In [1]:
from utils import load_corpus, load_stopwords, save_corpus
import pandas as pd
import os
from transformers import BertTokenizer, BertModel
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from sklearn import metrics
import warnings
import time

# BERT

In [ ]:
stopwords = load_stopwords("stopwords.txt")
TRAIN_PATH = "train2.txt"
TEST_PATH = "test2.txt"
train_data = load_corpus(TRAIN_PATH)
test_data = load_corpus(TEST_PATH)
df_train = pd.DataFrame(train_data, columns=["text", "label"])
df_test = pd.DataFrame(test_data, columns=["text", "label"])
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"
MODEL_PATH = "bert-base-chinese"
tokenizer = BertTokenizer.from_pretrained(MODEL_PATH)
bert = BertModel.from_pretrained(MODEL_PATH, ignore_mismatched_sizes=True)

Building prefix dict from the default dictionary ...
2025-05-20 11:03:51,556 - DEBUG - Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
2025-05-20 11:03:51,560 - DEBUG - Loading model from cache /tmp/jieba.cache
Loading model cost 1.345 seconds.
2025-05-20 11:03:52,905 - DEBUG - Loading model cost 1.345 seconds.
Prefix dict has been built successfully.
2025-05-20 11:03:52,907 - DEBUG - Prefix dict has been built successfully.


In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
bert = bert.to(device)
learning_rate = 1e-3
input_size = 768
num_epoches = 1
batch_size = 100
decay_rate = 0.9

In [ ]:
class MyDataset(Dataset):
    def __init__(self, df):
        self.data = df["text"].tolist()
        self.label = df["label"].tolist()

    def __getitem__(self, index):
        return self.data[index], self.label[index]

    def __len__(self):
        return len(self.label)
train_dataset = MyDataset(df_train)
test_dataset = MyDataset(df_test)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
class SentimentClassifier(nn.Module):
    def __init__(self, input_size):
        super(SentimentClassifier, self).__init__()
        self.fc = nn.Linear(input_size, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        out = self.fc(x)
        return self.sigmoid(out)
net = SentimentClassifier(input_size).to(device)
def evaluate(model, data_loader):
    model.eval()
    y_pred, y_true, y_prob = [], [], []
    
    with torch.no_grad():
        for texts, labels in data_loader:
            tokens = tokenizer(texts, 
                             padding=True, 
                             truncation=True, 
                             max_length=512,
                             return_tensors="pt")
            
            input_ids = tokens["input_ids"].to(device)
            attention_mask = tokens["attention_mask"].to(device)
            labels = labels.float().to(device)
            outputs = bert(input_ids, attention_mask=attention_mask)
            cls_embedding = outputs.last_hidden_state[:, 0, :]
            probas = model(cls_embedding).view(-1)
            preds = (probas > 0.5).long()
            y_pred.extend(preds.cpu().numpy())
            y_true.extend(labels.cpu().numpy())
            y_prob.extend(probas.cpu().numpy())


In [ ]:
# 训练函数
def train(model, train_loader, test_loader):
    criterion = nn.BCELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=decay_rate)
    for epoch in range(num_epoches):
        model.train()
        total_loss = 0
        
        for i, (texts, labels) in enumerate(train_loader):
            tokens = tokenizer(texts, 
                             padding=True, 
                             truncation=True, 
                             max_length=512,
                             return_tensors="pt")
            
            input_ids = tokens["input_ids"].to(device)
            attention_mask = tokens["attention_mask"].to(device)
            labels = labels.float().to(device)
            optimizer.zero_grad()
            with torch.no_grad():
                bert_outputs = bert(input_ids, attention_mask=attention_mask)
                cls_embedding = bert_outputs.last_hidden_state[:, 0, :]
            outputs = model(cls_embedding).view(-1)
            # 计算损失
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            # 反向传播
            loss.backward()
            optimizer.step()
            
            if (i+1) % 10 == 0:
                print(f"Epoch [{epoch+1}/{num_epoches}], Step [{i+1}/{len(train_loader)}], Loss: {total_loss/10:.4f}")
                total_loss = 0
        # 学习率衰减
        scheduler.step()
        # 评估
        print(f"\nEpoch {epoch+1} 评估结果:")
        evaluate(model, test_loader)
        # 保存模型
        model_path = f"./model/bert_dnn_{epoch+1}.model"
        torch.save(model.state_dict(), model_path)
        print(f"模型已保存到 {model_path}\n")
# 训练模型
start_time=time.time()
train(net, train_loader, test_loader)
time=time.time()-start_time
print(f"训练时间: {time:.2f}秒")

Epoch [1/1], Step [10/77], Loss: 0.6750
Epoch [1/1], Step [20/77], Loss: 0.5619
Epoch [1/1], Step [30/77], Loss: 0.4902
Epoch [1/1], Step [40/77], Loss: 0.4489
Epoch [1/1], Step [50/77], Loss: 0.4196
Epoch [1/1], Step [60/77], Loss: 0.4030
Epoch [1/1], Step [70/77], Loss: 0.3580

Epoch 1 评估结果:
模型已保存到 ./model/bert_dnn_1.model

训练时间: 40.63秒


In [8]:
# 预测函数
def predict_sentiment(texts, model_path="./model/bert_dnn_8.model"):
    # 加载模型
    model = SentimentClassifier(input_size).to(device)
    model.load_state_dict(torch.load(model_path))
    model.eval()
    
    # Tokenize输入
    tokens = tokenizer(texts, 
                      padding=True, 
                      truncation=True, 
                      max_length=512,
                      return_tensors="pt")
    
    input_ids = tokens["input_ids"].to(device)
    attention_mask = tokens["attention_mask"].to(device)
    
    # 预测
    with torch.no_grad():
        bert_outputs = bert(input_ids, attention_mask=attention_mask)
        cls_embedding = bert_outputs.last_hidden_state[:, 0, :]
        predictions = model(cls_embedding)
    
    # 返回结果
    results = []
    for text, prob in zip(texts, predictions.cpu().numpy()):
        sentiment = "正面" if prob > 0.5 else "负面"
        results.append(f"文本: {text}\n情感: {sentiment} (置信度: {prob[0]:.4f})")
    
    return "\n\n".join(results)

In [9]:
sample_texts = [
    "剧情老套，充满套路和硬凹的感动.",
    "食物份量十足，性价比超高，吃得很满足!",
]

print(predict_sentiment(sample_texts))

文本: 剧情老套，充满套路和硬凹的感动.
情感: 负面 (置信度: 0.0660)

文本: 食物份量十足，性价比超高，吃得很满足!
情感: 正面 (置信度: 0.9951)


/tmp/ipykernel_1834/604079376.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path))


# GPT

In [1]:
from transformers import BertTokenizer, GPT2LMHeadModel, TextGenerationPipeline
tokenizer = BertTokenizer.from_pretrained("./gpt2-chinese-cluecorpussmall")
model = GPT2LMHeadModel.from_pretrained("./gpt2-chinese-cluecorpussmall")
text_generator = TextGenerationPipeline(model, tokenizer)   
result = text_generator("在一个没有网络的世界里", max_length=500, do_sample=True)
print(result)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


[{'generated_text': '在一个没有网络的世界里 。 所 以 有 网 络 时 必 须 找 一 个 地 方 。 不 找 寻 一 个 真 实 的 自 我 ， 而 是 看 到 一 个 有 意 思 的 世 界 ， 而 不 是 只 是 感 官 上 的 想 法 。 如 果 这 个 世 界 是 虚 空 的 ， 那 么 大 人 物 就 是 现 实 的 ， 要 找 一 个 像 样 的 孩 子 、 长 辈 去 教 育 ， 你 就 是 虚 空 了 。 假 如 他 认 为 自 己 真 的 虚 空 了 ， 我 想 他 也 不 会 这 么 做 。 说 到 这 里 的 时 候 ， 王 小 波 和 张 嘉 佳 ， 他 们 都 还 有 一 些 不 成 熟 的 想 法 ， 但 他 们 在 《 小 波 沉 思 录 》 中 说 ， 虚 空 的 理 由 不 就 是 虚 空 吗 ？ 不 就 是 说 我 不 存 在 什 么 东 西 吗 ？ 王 小 波 说 ， 自 己 可 能 有 些 感 伤 ， 甚 至 有 些 悲 痛 ， 毕 竟 自 己 不 是 从 小 就 接 受 这 种 教 育 ， 我 只 是 喜 欢 这 个 世 界 ， 我 觉 得 在 现 实 工 作 中 我 们 能 够 做 的 事 情 真 的 非 常 多 。 如 果 这 个 世 界 上 存 在 什 么 东 西 ， 这 个 世 界 就 真 的 没 有 意 义 。 如 果 世 界 上 的 意 义 在 于 虚 空 ， 那 么 大 人 物 就 是 现 实 的 ， 要 找 一 个 像 样 的 孩 子 、 长 辈 ， 你 就 是 现 实 的 、 有 意 思 的 ， 要 去 教 育 孩 子 和 长 辈 。 有 孩 子 的 王 小 波 说 ， 在 我 们 的 世 界 观 中 ， 那 些 没 有 网 络 的 世 界 是 虚 空 的 ， 没 有 家 的 时 候 是 虚 空 的 。 在 有 孩 子 的 世 界 里 ， 孩 子 真 的 能 够 有 空 吗 ？ 王 小 波 解 释 说 ， 没 有 孩 子 没 法 有 精 神 世 界 ， 没 有 精 神 世 界 有 精 神 世 界 ， 没 有 精 神 世 界 。 这 时 候 家 庭 就 不 是 用 来 教 授 知 识 的 地 方 ， 不 仅 仅 是 学 分 的 问 题'}]
